## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
def find_candle_trend (candle_no):
    if candle_no < 10:
        return 0
    
    data = pd.read_csv(filename)
    data = data.iloc[candle_no-5:candle_no+1]
#    display (data[['Open','Close','Low','High']])
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = int(r2_score(Y, y_pred)*100)
#    print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)
    
    if coeficient > 0:
        Current_Market = 1 *  Current_Market_Fit## Bullish / Buy ##
    else:
        Current_Market = -1 * Current_Market_Fit ## Bearish / Sell ##
    
    return Current_Market

In [5]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [6]:
print('Instrument:', instrument)

Instrument: EUR_USD


### The Configs for Run:

In [7]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [8]:
Load_10K_Records=True
Load_20K_Records=False

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [9]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [10]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [11]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [12]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
        response=second_response 
else:
    response=first_response['candles']

In [13]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [15]:
data.shape

(9670, 12)

In [16]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-11-30,18:00:00,18,334,0,Monday,2015-11-30T18:00:00.000000000Z,2459,1.05602,1.05760,1.05592,1.05644
1,2015-11-30,22:00:00,22,334,0,Monday,2015-11-30T22:00:00.000000000Z,1741,1.05666,1.05819,1.05634,1.05781
2,2015-12-01,02:00:00,2,335,1,Tuesday,2015-12-01T02:00:00.000000000Z,3487,1.05785,1.05968,1.05714,1.05818
3,2015-12-01,06:00:00,6,335,1,Tuesday,2015-12-01T06:00:00.000000000Z,5109,1.05819,1.06193,1.05778,1.05980
4,2015-12-01,10:00:00,10,335,1,Tuesday,2015-12-01T10:00:00.000000000Z,3789,1.05976,1.06116,1.05856,1.06062


In [17]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9665,2022-02-17,02:00:00,2,48,3,Thursday,2022-02-17T02:00:00.000000000Z,35305,1.13818,1.13850,1.13232,1.13564
9666,2022-02-17,06:00:00,6,48,3,Thursday,2022-02-17T06:00:00.000000000Z,48292,1.13566,1.13863,1.13428,1.13614
9667,2022-02-17,10:00:00,10,48,3,Thursday,2022-02-17T10:00:00.000000000Z,33519,1.13614,1.13795,1.13517,1.13595
9668,2022-02-17,14:00:00,14,48,3,Thursday,2022-02-17T14:00:00.000000000Z,49834,1.13594,1.13796,1.13507,1.13594
9669,2022-02-17,18:00:00,18,48,3,Thursday,2022-02-17T18:00:00.000000000Z,5790,1.13593,1.13647,1.13571,1.13630


## Simple Moving Average (SMA)

In [18]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [19]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [20]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [21]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9665,2022-02-17,02:00:00,2,48,3,Thursday,2022-02-17T02:00:00.000000000Z,35305,1.13818,1.13850,1.13232,1.13564,1.1370,1.1365,1.1347,-0.00136,-0.00086,0.00094
9666,2022-02-17,06:00:00,6,48,3,Thursday,2022-02-17T06:00:00.000000000Z,48292,1.13566,1.13863,1.13428,1.13614,1.1369,1.1365,1.1347,-0.00076,-0.00036,0.00144
9667,2022-02-17,10:00:00,10,48,3,Thursday,2022-02-17T10:00:00.000000000Z,33519,1.13614,1.13795,1.13517,1.13595,1.1367,1.1365,1.1347,-0.00075,-0.00055,0.00125
9668,2022-02-17,14:00:00,14,48,3,Thursday,2022-02-17T14:00:00.000000000Z,49834,1.13594,1.13796,1.13507,1.13594,1.1364,1.1366,1.1350,-0.00046,-0.00066,0.00094
9669,2022-02-17,18:00:00,18,48,3,Thursday,2022-02-17T18:00:00.000000000Z,5790,1.13593,1.13647,1.13571,1.13630,1.1360,1.1367,1.1352,0.00030,-0.00040,0.00110


## Price Range

In [22]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [23]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [24]:
data['Trend'] = 0

In [25]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5,Trend
0,2015-12-03,22:00:00,22,337,3,Thursday,2015-12-03T22:00:00.000000000Z,5587,1.09414,1.09566,...,0.00574,0.00466,-0.00108,0,0.00314,0.00466,0.00422,-0.00373,-0.00778,0
1,2015-12-04,02:00:00,2,338,4,Friday,2015-12-04T02:00:00.000000000Z,3762,1.09104,1.09464,...,0.00418,0.00074,-0.00344,1,-0.00286,0.00074,0.00058,0.00314,-0.00373,0
2,2015-12-04,06:00:00,6,338,4,Friday,2015-12-04T06:00:00.000000000Z,10055,1.09388,1.09452,...,0.00865,0.00553,-0.00312,0,0.00489,0.00553,0.00801,-0.00286,0.00314,0
3,2015-12-04,10:00:00,10,338,4,Friday,2015-12-04T10:00:00.000000000Z,14955,1.08901,1.09507,...,0.00952,0.00249,-0.00703,1,-0.00357,0.00249,0.00346,0.00489,-0.00286,0
4,2015-12-04,14:00:00,14,338,4,Friday,2015-12-04T14:00:00.000000000Z,18128,1.09256,1.09508,...,0.01145,0.00824,-0.00321,0,0.00572,0.00824,0.00893,-0.00357,0.00489,0


In [26]:
data.iloc

In [27]:
data.index

RangeIndex(start=0, stop=9651, step=1)

In [30]:
for candle_no in range (len(data.index.tolist())):
    data["Trend"] = find_candle_trend(candle_no)

In [33]:
data.head(50)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5,Trend
0,2015-12-03,22:00:00,22,337,3,Thursday,2015-12-03T22:00:00.000000000Z,5587,1.09414,1.09566,...,0.00574,0.00466,-0.00108,0,0.00314,0.00466,0.00422,-0.00373,-0.00778,-25
1,2015-12-04,02:00:00,2,338,4,Friday,2015-12-04T02:00:00.000000000Z,3762,1.09104,1.09464,...,0.00418,0.00074,-0.00344,1,-0.00286,0.00074,0.00058,0.00314,-0.00373,-25
2,2015-12-04,06:00:00,6,338,4,Friday,2015-12-04T06:00:00.000000000Z,10055,1.09388,1.09452,...,0.00865,0.00553,-0.00312,0,0.00489,0.00553,0.00801,-0.00286,0.00314,-25
3,2015-12-04,10:00:00,10,338,4,Friday,2015-12-04T10:00:00.000000000Z,14955,1.08901,1.09507,...,0.00952,0.00249,-0.00703,1,-0.00357,0.00249,0.00346,0.00489,-0.00286,-25
4,2015-12-04,14:00:00,14,338,4,Friday,2015-12-04T14:00:00.000000000Z,18128,1.09256,1.09508,...,0.01145,0.00824,-0.00321,0,0.00572,0.00824,0.00893,-0.00357,0.00489,-25
5,2015-12-04,18:00:00,18,338,4,Friday,2015-12-04T18:00:00.000000000Z,5358,1.08680,1.08851,...,0.00339,0.00028,-0.00311,1,-0.00143,0.00028,0.00168,0.00572,-0.00357,-25
6,2015-12-06,22:00:00,22,340,6,Sunday,2015-12-06T22:00:00.000000000Z,3152,1.08748,1.08874,...,0.00242,0.00196,-0.00046,0,0.00070,0.00196,0.00116,-0.00143,0.00572,-25
7,2015-12-07,02:00:00,2,341,0,Monday,2015-12-07T02:00:00.000000000Z,1282,1.08674,1.08697,...,0.00154,0.00085,-0.00069,0,0.00062,0.00085,0.00131,0.00070,-0.00143,-25
8,2015-12-07,06:00:00,6,341,0,Monday,2015-12-07T06:00:00.000000000Z,5482,1.08614,1.08728,...,0.00614,0.00557,-0.00057,0,0.00443,0.00557,0.00500,0.00062,0.00070,-25
9,2015-12-07,10:00:00,10,341,0,Monday,2015-12-07T10:00:00.000000000Z,5023,1.08168,1.08261,...,0.00299,0.00087,-0.00212,1,-0.00006,0.00087,0.00206,0.00443,0.00062,-25


In [ ]:
#vols = data.index[data['Volume']>5500].tolist()
vols = data.index.tolist()
len (vols)

In [ ]:
vols = vols [9641:]

In [ ]:
len(vols)

In [ ]:
vols